In [ ]:
import os

In [ ]:
os.environ.keys()

# Imports

In [ ]:
import pygrib
import os
import numpy as np

from matplotlib import pyplot as plt
from matplotlib import colors
from mpl_toolkits.basemap import Basemap, addcyclic

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.figure_factory as ff

import ipywidgets

init_notebook_mode(connected=True)

In [ ]:
import mpl_toolkits
mpl_toolkits.__version__

In [ ]:
%matplotlib inline

# Define functions

## Define a function that takes a list of pygrib.gribmessage objects and returns a dictionary, where the key 'data' contains a 3D tensor with data from the pygrib.gribmessage objects

In [ ]:
def grb_to_grid(grb_obj):
    """Takes a single grb object containing multiple
    levels. Assumes same time, pressure levels. Compiles to a cube"""
    n_levels = len(grb_obj)
    levels = np.array([grb_element['level'] for grb_element in grb_obj])
    indexes = np.argsort(levels)[::-1] # highest pressure first
    cube = np.zeros([n_levels, grb_obj[0].values.shape[0], grb_obj[1].values.shape[1]])
    for i in range(n_levels):
        cube[i,:,:] = grb_obj[indexes[i]].values
    cube_dict = {'data' : cube, 'units' : grb_obj[0]['units'],
                 'levels' : levels[indexes]}
    return cube_dict

### Define functions to return outlines of countries and coastlines

In [ ]:
# Make trace-generating function (return a Scatter object)
def make_scatter(x,y):
    return go.Scatter(
        x=x,
        y=y,
        mode='lines',
        line=go.scatter.Line(color="black"),
        name=' '  # no name on hover
    )

# Functions converting coastline/country polygons to lon/lat traces
def polygons_to_traces(poly_paths, N_poly, basemap_map):
    ''' 
    pos arg 1. (poly_paths): paths to polygons
    pos arg 2. (N_poly): number of polygon to convert
    '''
    traces = []  # init. plotting list 

    for i_poly in range(N_poly):
        poly_path = poly_paths[i_poly]
        
        # get the Basemap coordinates of each segment
        coords_cc = np.array(
            [(vertex[0],vertex[1]) 
             for (vertex,code) in poly_path.iter_segments(simplify=False)]
        )
        
        # convert coordinates to lon/lat by 'inverting' the Basemap projection
        lon_cc, lat_cc = basemap_map(coords_cc[:,0],coords_cc[:,1], inverse=True)
        
        # add plot.ly plotting options
        traces.append(make_scatter(lon_cc,lat_cc))
     
    return traces

# Function generating coastline lon/lat traces
def get_coastline_traces(basemap_map):
    poly_paths = basemap_map.drawcoastlines().get_paths() # coastline polygon paths
    N_poly = 91 # use only the 91st biggest coastlines (i.e. no rivers)
    return polygons_to_traces(poly_paths, N_poly, basemap_map)

# Function generating country lon/lat traces
def get_country_traces(basemap_map):
    poly_paths = basemap_map.drawcountries().get_paths() # country polygon paths
    N_poly = len(poly_paths)  # use all countries
    return polygons_to_traces(poly_paths, N_poly, basemap_map)


### Define functions to read data and return plotly FigureWidget object

In [ ]:
def get_data_as_cube_and_lats_lons(file_path, file_name, data_type):
    grbs = pygrib.open(os.path.join(file_path, file_name))
    grbs.messages
    grb = grbs.readline()
    lats,lons = grb.latlons()
    grb_data = grbs.select(name=data_type)
    grb_cube_data=grb_to_grid(grb_data)
    
    forecast_date = grb.validDate
    issuance_date = grb.analDate
    grbs.close()
    
    return grb_cube_data, lats, lons, forecast_date, issuance_date

def rotate_data_and_lats(grb_matrix_data, lons, degrees=180):
    lons_rotated = lons.copy()
    lons_rotated[lons >= degrees] = lons[lons >= degrees] - (2*degrees)

    # proper rotation obtained from: https://plot.ly/ipython-notebooks/basemap-maps/
    i_east = lons_rotated[0, :] >= 0  # indices of east lon
    i_west = lons_rotated[0, :] < 0   # indices of west lon
    
    # stack the two halves
    lons_rotated = np.hstack((lons_rotated[:, i_west], lons_rotated[:, i_east]))  

    # Correspondingly, shift the data array
    data_rotated = np.hstack((grb_matrix_data[:,i_west], grb_matrix_data[:,i_east]))
    
    return data_rotated, lons_rotated

def get_rotated_basemap(degrees=180):
    m_rotated = Basemap(llcrnrlon = 1 - degrees, llcrnrlat = -89, urcrnrlon = degrees,
           urcrnrlat = 89 , projection = 'mill', area_thresh =10000,
           resolution='l')
    
    return m_rotated
    
def get_widget_figure(file_path, file_name, data_type, degrees_to_rotate=180):
    grb_cube_data, lats, lons, forecast_date, issuance_date = get_data_as_cube_and_lats_lons(
        file_path, file_name, data_type)
    matrix_data = grb_cube_data['data'][10]
    data_rotated, lons_rotated = rotate_data_and_lats(matrix_data, lons)
    basemap_rotated = get_rotated_basemap()
    
    anno_text = str(forecast_date) + '<br>(issued on ' + str(issuance_date) + ')'
    #"Data courtesy of
    #<a href='http://www.esrl.noaa.gov/psd/data/composites/day/'>\
    #NOAA Earth System Research Laboratory</a>"

    axis_style = dict(
        zeroline=False,
        showline=False,
        showgrid=False,
        ticks='',
        showticklabels=False,
    )

    layout1 = go.Layout(
        title=go.layout.Title(
        text=data_type,
        xref='paper',
        x=0
        ),
        showlegend=False,
        hovermode="closest", # highlight closest point on hover
        margin = {'t':100, 'b':60, 'l':60, 'r':0, 'pad':8}, 
        xaxis=go.layout.XAxis(
            axis_style,
            range = [-degrees_to_rotate, degrees_to_rotate]  # restrict y-axis to range of lon
        ),
        yaxis=go.layout.YAxis(
            axis_style,
            range=[-85, 85]
        ),
        annotations=[
            dict(
                text=anno_text,
                xref='paper',
                yref='paper',
                x=0,
                y=1,
                yanchor='bottom',
                showarrow=False,
                align='left'
            )
        ],
        autosize=False,
        width=500,
        height=400
    )
    
    traces_cc = get_coastline_traces(basemap_rotated)+get_country_traces(basemap_rotated)
    
    fw = go.FigureWidget(data=traces_cc + [go.Contour(z=data_rotated, x=lons_rotated[0, :], 
                                                      y=lats[:, 0])], layout=layout1)
    return fw



# Define data path

In [ ]:
data_folder = os.path.join(os.path.expanduser('~'), 
                           'Documents', 'Projects', 'WeatherWind', 'data')

# Plots side-by-side

In [ ]:
# downloaded from https://nomads.ncdc.noaa.gov/data/gfs-avn-hi/201805/20180508/
data_file = 'gfs_3_20180508_0600_027.grb2' 

In [ ]:
fw1 = get_widget_figure(data_folder, data_file, "V component of wind")
fw2 = get_widget_figure(data_folder, data_file, "U component of wind")

In [ ]:
# https://community.plot.ly/t/plotly-subplots-using-fig-objects-instead-of-traces/11969/4
# https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Styling.html
fig_subplots = ipywidgets.VBox([fw1, fw2], layout=ipywidgets.Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    width='100%'))

In [ ]:
# u is the ZONAL VELOCITY, i.e. the component of the horizontal wind TOWARDS EAST. 
# v is the MERIDIONAL VELOCITY, i.e. the component of the horizontal wind TOWARDS NORTH
# http://tornado.sfsu.edu/geosciences/classes/m430/Wind/WindDirection.html

fig_subplots